# Understanding Browser Fingerprinting

Browser fingerprinting is a way of recognizing various attributes of a browser using the specific feature information like the type of font used or even a trivial thing like the type of emoji. It has its own pros and cons: If the goal is to protect uniqueness of a individual then it works fine in the case when an individual needs to be recognized to eliminate foul play but if the goal is mere advertisements then it can affect the privacy of a individual in a negative aspect. There have been various projects related to this genre which have worked to find which features are most important to recognize a specific website. Some of them which I went through are Panopticlick, AmIUnique.org, slido and Erik Flood and Joel Karlsson. Browser fingerprinting. 2012.



In [51]:
from dask.distributed import Client,progress
Client()


Client Scheduler: tcp://127.0.0.1:5062 Dashboard: http://127.0.0.1:8787,Cluster Workers: 4 Cores: 4 Memory: 8.50 GB


In [52]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


In [53]:
import tldextract

#DATA_DIR = 'sample_10percent_value_1000_only.parquet'

def extract_domain(url):
    """Use tldextract to return the base domain from a url"""
    try:
        extracted = tldextract.extract(url)
        return '{}.{}'.format(extracted.domain, extracted.suffix)
    except Exception as e:
        return 'ERROR'

# Feature Evaluation
Based on my learning's I decided to further dive in features : " location, script-url, symbol and value1000'. On the basis of initial analysis I could understand that we can extract additional features from the script-url attribute with respect to symbol and value attribute. As various rows of location contained same urls but different symbol values which could uncover various important elements like font used through canvas finger printing , audio finger printing details , and if a script could have been blocked by adblock.



In [54]:
df = dd.read_parquet('sample_10percent_value_1000_only.parquet', engine='pyarrow',columns=['argument_0', 'func_name','in_iframe', 'location', 'operation','script_url', 'symbol', 'time_stamp', 'value_1000'])

In [55]:
df.head()


,argument_0,func_name,in_iframe,location,operation,script_url,symbol,time_stamp,value_1000
0,None,w.fastXDM.Client,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/api/xdm.js?1449919642,window.name,2017-12-16 19:02:31.406,fXDcab74
1,None,w.fastXDM.Client,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/api/xdm.js?1449919642,window.name,2017-12-16 19:02:31.407,fXDcab74
2,None,,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/al/aes_light.js?592436914,window.navigator.userAgent,2017-12-16 19:02:31.659,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...
3,None,,True,https://pos.baidu.com/s?hei=70&wid=670&di=u313...,get,https://cpro.baidustatic.com/cpro/ui/noexpire/...,window.navigator.userAgent,2017-12-16 00:24:09.355,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...
4,None,Fd.iterate,False,http://serienjunkies.org/smilf/smilf-season-1-...,get,https://apis.google.com/js/plusone.js?_=151338...,window.document.cookie,2017-12-16 01:24:30.372,_ga=GA1.2.1529583939.1513387469; _gid=GA1.2.17...


In [56]:
import pandas as pd
import numpy as np
import matplotlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso


In [57]:
from urllib.parse import urlparse

EMPTY_STRING = 'EMPTY_STRING'


def get_netloc(x):
    p = urlparse(x)
    val = p.netloc
    if len(val) == 0:
        val = EMPTY_STRING
    return val


def get_path(x):
    p = urlparse(x)
    val = p.path
    if len(val) == 0:
        val = EMPTY_STRING
    return val


def get_end_of_path(x):
    splits = x.split('/')
    val = ''
    if len(splits) > 0:
        val = splits[-1]
    else:
        val = x
    if len(val) == 0:
        val = EMPTY_STRING
    return val


def get_clean_script(x):
    p = urlparse(x)
    return f'{p.netloc}{p.path}'


In [58]:
df['script_netloc'] = df.script_url.apply(get_netloc, meta=('O'))
df['script_path'] = df.script_url.apply(get_path, meta=('O'))
df['script_path_end'] = df.script_path.apply(get_end_of_path, meta=('O'))
df['agg'] = df.script_netloc + '||' + df.script_path_end + '||' + df.func_name
df['location_domain'] = df.location.apply(extract_domain, meta=('x', 'str'))

df['location_base_url'] = df.location.apply(get_netloc, meta=('x', 'str'))
df['script_domain'] = df.script_url.apply(extract_domain, meta=('x', 'str'))




In [59]:
df.head()

,argument_0,func_name,in_iframe,location,operation,script_url,symbol,time_stamp,value_1000,script_netloc,script_path,script_path_end,agg,location_domain,location_base_url,script_domain
0,None,w.fastXDM.Client,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/api/xdm.js?1449919642,window.name,2017-12-16 19:02:31.406,fXDcab74,vk.com,/js/api/xdm.js,xdm.js,vk.com||xdm.js||w.fastXDM.Client,vk.com,vk.com,vk.com
1,None,w.fastXDM.Client,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/api/xdm.js?1449919642,window.name,2017-12-16 19:02:31.407,fXDcab74,vk.com,/js/api/xdm.js,xdm.js,vk.com||xdm.js||w.fastXDM.Client,vk.com,vk.com,vk.com
2,None,,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/al/aes_light.js?592436914,window.navigator.userAgent,2017-12-16 19:02:31.659,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,vk.com,/js/al/aes_light.js,aes_light.js,vk.com||aes_light.js||,vk.com,vk.com,vk.com
3,None,,True,https://pos.baidu.com/s?hei=70&wid=670&di=u313...,get,https://cpro.baidustatic.com/cpro/ui/noexpire/...,window.navigator.userAgent,2017-12-16 00:24:09.355,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,cpro.baidustatic.com,/cpro/ui/noexpire/js/4.0.0/adClosefeedbackUpgr...,adClosefeedbackUpgrade.min.js,cpro.baidustatic.com||adClosefeedbackUpgrade.m...,baidu.com,pos.baidu.com,baidustatic.com
4,None,Fd.iterate,False,http://serienjunkies.org/smilf/smilf-season-1-...,get,https://apis.google.com/js/plusone.js?_=151338...,window.document.cookie,2017-12-16 01:24:30.372,_ga=GA1.2.1529583939.1513387469; _gid=GA1.2.17...,apis.google.com,/js/plusone.js,plusone.js,apis.google.com||plusone.js||Fd.iterate,serienjunkies.org,serienjunkies.org,google.com


In [60]:
df['symbol_parts'] = df.symbol.str.split('.')
df['symbol_0'] = df.symbol_parts.str.get(0)
df['symbol_1'] = df.symbol_parts.str.get(1)
df['symbol_2'] = df.symbol_parts.str.get(2)
df.head()

,argument_0,func_name,in_iframe,location,operation,script_url,symbol,time_stamp,value_1000,script_netloc,script_path,script_path_end,agg,location_domain,location_base_url,script_domain,symbol_parts,symbol_0,symbol_1,symbol_2
0,None,w.fastXDM.Client,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/api/xdm.js?1449919642,window.name,2017-12-16 19:02:31.406,fXDcab74,vk.com,/js/api/xdm.js,xdm.js,vk.com||xdm.js||w.fastXDM.Client,vk.com,vk.com,vk.com,"[window, name]",window,name,NaN
1,None,w.fastXDM.Client,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/api/xdm.js?1449919642,window.name,2017-12-16 19:02:31.407,fXDcab74,vk.com,/js/api/xdm.js,xdm.js,vk.com||xdm.js||w.fastXDM.Client,vk.com,vk.com,vk.com,"[window, name]",window,name,NaN
2,None,,True,https://vk.com/widget_comments.php?app=2297596...,get,https://vk.com/js/al/aes_light.js?592436914,window.navigator.userAgent,2017-12-16 19:02:31.659,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,vk.com,/js/al/aes_light.js,aes_light.js,vk.com||aes_light.js||,vk.com,vk.com,vk.com,"[window, navigator, userAgent]",window,navigator,userAgent
3,None,,True,https://pos.baidu.com/s?hei=70&wid=670&di=u313...,get,https://cpro.baidustatic.com/cpro/ui/noexpire/...,window.navigator.userAgent,2017-12-16 00:24:09.355,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,cpro.baidustatic.com,/cpro/ui/noexpire/js/4.0.0/adClosefeedbackUpgr...,adClosefeedbackUpgrade.min.js,cpro.baidustatic.com||adClosefeedbackUpgrade.m...,baidu.com,pos.baidu.com,baidustatic.com,"[window, navigator, userAgent]",window,navigator,userAgent
4,None,Fd.iterate,False,http://serienjunkies.org/smilf/smilf-season-1-...,get,https://apis.google.com/js/plusone.js?_=151338...,window.document.cookie,2017-12-16 01:24:30.372,_ga=GA1.2.1529583939.1513387469; _gid=GA1.2.17...,apis.google.com,/js/plusone.js,plusone.js,apis.google.com||plusone.js||Fd.iterate,serienjunkies.org,serienjunkies.org,google.com,"[window, document, cookie]",window,document,cookie


Using a specific location, I tried to find out various features that can be extracted from the symbol column like plugin, userAgent, language and other details. 
Some arguments have "Cwm fjordbank glyphs vext quiz" which is mentioned in various paper to be associated with canvas fingerprinting. This is said to be used to uniquely identify a browser almost accurately. If a mobile phone is considered , emojis arev also considered as a significant trait to know about a browser as every mobile has unique way of representing its emojis. 

In [38]:
df.loc[df["location"] == "https://www.johnlewis.com/browse/gifts/gift-food-wine-champagne/view-all-hampers/_/N-amr", ("symbol_2")].compute()

0                            NaN
1                        getItem
2                      userAgent
3                            NaN
4                        getItem
5                            NaN
6                        getItem
7                      userAgent
8                            NaN
9                            NaN
10                        cookie
11                     userAgent
12                        cookie
13                        cookie
14                        cookie
15                        cookie
16                        cookie
17                        cookie
18                        cookie
19                     userAgent
20                     userAgent
21                        cookie
22                     userAgent
23                      platform
24      plugins[Shockwave Flash]
25                     userAgent
26                        cookie
27                     userAgent
28                       appName
29                     userAgent
          

Further, using the string operations mentioned in issue_36 we can find significant factors that could tell us how a browser is recognized
- location : http://ca.puma.com/en_CA/kids	
- window.navigator.language : en-US
- window.navigator.plugins[Shockwave Flash].name: Shockwave Flash 28.0 r0
- window.Storage.length : 0
- window.screen.pixelDepth :24
- window.screen.colorDepth :24
- window.navigator.platform :Linux x86_64
- window.navigator.doNotTrack :unspecified
- window.navigator.userAgent: Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0
- Time_stamp	: 12/16/2017  3:46:19 AM

## Audio fingerprint aspect

OfflineAudioContex/AudioContext
- createOscillator
- createDynamicsCompressor
- destination
- startRendering
- oncomplete


## Canvas fingerprinting aspect
CanvasRenderingContext2D
- font
- getImagedata
- Fill/fillstyle
- globalcompositionoperation

HTMLCanvasElement
- Height,width,style,draggable


In [27]:
df.to_csv('E:/fingerp/new.*.csv')

['E:/fingerp/new.000.csv',
 'E:/fingerp/new.001.csv',
 'E:/fingerp/new.002.csv',
 'E:/fingerp/new.003.csv',
 'E:/fingerp/new.004.csv',
 'E:/fingerp/new.005.csv',
 'E:/fingerp/new.006.csv',
 'E:/fingerp/new.007.csv',
 'E:/fingerp/new.008.csv',
 'E:/fingerp/new.009.csv',
 'E:/fingerp/new.010.csv',
 'E:/fingerp/new.011.csv',
 'E:/fingerp/new.012.csv',
 'E:/fingerp/new.013.csv',
 'E:/fingerp/new.014.csv',
 'E:/fingerp/new.015.csv',
 'E:/fingerp/new.016.csv',
 'E:/fingerp/new.017.csv',
 'E:/fingerp/new.018.csv',
 'E:/fingerp/new.019.csv',
 'E:/fingerp/new.020.csv',
 'E:/fingerp/new.021.csv',
 'E:/fingerp/new.022.csv',
 'E:/fingerp/new.023.csv',
 'E:/fingerp/new.024.csv',
 'E:/fingerp/new.025.csv',
 'E:/fingerp/new.026.csv',
 'E:/fingerp/new.027.csv',
 'E:/fingerp/new.028.csv',
 'E:/fingerp/new.029.csv',
 'E:/fingerp/new.030.csv',
 'E:/fingerp/new.031.csv',
 'E:/fingerp/new.032.csv',
 'E:/fingerp/new.033.csv',
 'E:/fingerp/new.034.csv',
 'E:/fingerp/new.035.csv',
 'E:/fingerp/new.036.csv',
 

In [61]:
featureval_df = df[df.location == 'https://www.johnlewis.com/browse/gifts/gift-food-wine-champagne/view-all-hampers/_/N-amr']
fv_urls = featureval_df.symbol.unique().persist()
progress(fv_urls, notebook=False)


[########################################] | 100% Completed |  7min 37.8s

## Plugins and Fonts
According to some papers, the plugins with list of fonts are  very significant in order to identify a browser. For example, Iphone is considered to be least affected by browser fingerprinting as it has no flash setting. Hence Desktop machines are more vulnerable to this than mobile browsers.

# Entropy
I have used Excel for most of my analysis due to the size of file and memory limitation on my side. That being said , I have mostly focused on location , symbol and value feature which could sufficiently identify a browser on the basis of its font type, user-agent, Date format , platform ,Canvas and audio fingerprint. I plan to use entropy to find the feature combination that is best to identify a browser. We still have the question of which urls could be blocked by adblocks? I decided to consider in_iframe , location , operation and script_url attributes to scratch the surface of the problem. The heuristic here is to check whether the feature in_iframe is true and script_url of a certain location has a '.js' file with similar '.js' file blocked in the past. Can we build a model to predict if a script would be blocked by adblock or not? 
We could consider features with lowest entropy and train the model based on the past data.